In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from ipywidgets import IntProgress
from IPython.display import display

p = IntProgress()
display(p)

train = pd.read_csv(r"C:\Users\hg\Desktop\MLfinalproject\train.csv")
train_label = pd.read_csv(r"C:\Users\hg\Desktop\MLfinalproject\train_label.csv")
test = pd.read_csv(r"C:\Users\hg\Desktop\MLfinalproject\test.csv")

train_null = train.isnull()

for i in range(train.shape[0]):
    if train_null["company"][i] == False and train_null["agent"][i] == True:
        train["agent"][i] = train["company"][i]

train_filled_value = train.fillna({"children": 0, "country": "PRT","agent":9.0})

test_null = test.isnull()

for i in range(test.shape[0]):
    if test_null["company"][i] == False and test_null["agent"][i] == True:
        test["agent"][i] = test["company"][i]

test_filled_value = test.fillna({"country": "PRT","agent":9.0})

IntProgress(value=0)

<ipython-input-1-46e996aa58ac>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["agent"][i] = train["company"][i]
<ipython-input-1-46e996aa58ac>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["agent"][i] = test["company"][i]


In [2]:
train_filled_value = train_filled_value.drop(["ID","hotel","is_canceled","company","adr","reservation_status","reservation_status_date"], axis=1)
test_filled_value = test_filled_value.drop(["ID","hotel","company"], axis=1)

df_alldata = pd.concat([train_filled_value, test_filled_value], axis=0)
df_alldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119390 entries, 0 to 27858
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   lead_time                       119390 non-null  int64  
 1   arrival_date_year               119390 non-null  int64  
 2   arrival_date_month              119390 non-null  object 
 3   arrival_date_week_number        119390 non-null  int64  
 4   arrival_date_day_of_month       119390 non-null  int64  
 5   stays_in_weekend_nights         119390 non-null  int64  
 6   stays_in_week_nights            119390 non-null  int64  
 7   adults                          119390 non-null  int64  
 8   children                        119390 non-null  float64
 9   babies                          119390 non-null  int64  
 10  meal                            119390 non-null  object 
 11  country                         119390 non-null  object 
 12  market_segment   

In [81]:
category_column=["ID","hotel","is_canceled","arrival_date_year","arrival_date_month","meal","country","market_segment",
                 "distribution_channel","is_repeated_guest","reserved_room_type","assigned_room_type","deposit_type",
                "agent","customer_type","reservation_status","reservation_status_date"]
numerical_column=["lead_time","arrival_date_week_number","arrival_date_day_of_month","stays_in_weekend_nights",
                 "stays_in_week_nights","adults","children","babies","previous_cancellations","previous_bookings_not_canceled",
                 "booking_changes","days_in_waiting_list","required_car_parking_spaces","total_of_special_requests"]

category_column_new=[]
numerical_column_new=[]
for i in df_alldata:
    if i in category_column:
        category_column_new.append(i)
    if i in numerical_column:
        numerical_column_new.append(i)
for j in category_column_new:
    df_alldata[j] = df_alldata[j].astype('str')
df_numerical = df_alldata.loc[:,numerical_column_new]
df_category = df_alldata.loc[:,category_column_new]
df_category = pd.get_dummies(df_category)
df2 = pd.concat( [df_category, df_numerical], axis=1 )

df_train=df2[:91531]
df_test=df2[91531:]
#df_test.info()

df_train_numerical=df_numerical[:91531]
df_test_numerical=df_numerical[91531:]
df_train_category=df_category[:91531]
df_test_category=df_category[91531:]

In [34]:
df_train.head()

,arrival_date_year_2015,arrival_date_year_2016,arrival_date_year_2017,arrival_date_month_April,arrival_date_month_August,arrival_date_month_December,arrival_date_month_February,arrival_date_month_January,arrival_date_month_July,arrival_date_month_June,...,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests
0,1,0,0,0,0,0,0,0,1,0,...,0,2,0.0,0,0,0,3,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,2,1,0.0,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,2,2,0.0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,...,2,2,0.0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,...,2,2,0.0,0,0,0,0,0,0,0


In [5]:
x = df_train
y = train["is_canceled"]
y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

#RandomForest
rfc=RandomForestClassifier()
#rfc=RandomForestClassifier(n_estimators=100,n_jobs = -1,random_state =50, min_samples_leaf = 10)
rfc.fit(X_train,y_train)
y_iscanceled_predict_train = rfc.predict(X_train)
y_iscanceled_predict_test = rfc.predict(X_test)
score_rfc = rfc.score(X_test,y_test)
print("Random Forest Accuracy = ",score_rfc*100," %")
y_iscanceled_predict = np.concatenate((y_iscanceled_predict_train,y_iscanceled_predict_test),axis=0)

Random Forest Accuracy =  90.61014912328618  %


In [25]:
%%time
#Logistic
from sklearn import preprocessing, linear_model
logistic_regr = linear_model.LogisticRegression()
logistic_regr.fit(X_train,y_train)
survived_predictions = logistic_regr.predict(X_test)
accuracy = logistic_regr.score(X_test,y_test)
print("Logistic Regression Accuracy = ",accuracy*100,"%")

Logistic Regression Accuracy =  83.34061612409876 %
Wall time: 11.3 s


c:\users\hg\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
#liblinear predict is_canceled
import requests
import sys
import os
os.environ["PATH"] = 'C:\\Windows\\System32\\' + os.pathsep + os.environ["PATH"]
sys.path.append('C:\\Users\\hg\\Desktop\\liblinear-2.42\\windows')
sys.path.append('C:\\Users\\hg\\Desktop\\liblinear-2.42\\python')
from liblinearutil import *
x =np.array(x)
prob  = problem(y, x)
param = parameter('-s 0 -c 50 -e 0.000001')
m = train(prob, param)
p_label_predict, p_acc, p_val = predict(y, x, m)

Accuracy = 84.508% (77351/91531) (classification)


In [6]:
from sklearn.preprocessing import StandardScaler
category_column_1hot = []
for k in df_train:
    if k not in category_column_new:
        category_column_1hot.append(k)

x_train_std = StandardScaler().fit_transform(df_train[numerical_column_new])
x_train_std = np.concatenate((df_train[category_column_1hot], x_train_std), axis=1)

x_test_std = StandardScaler().fit_transform(df_test[numerical_column_new])
x_test_std = np.concatenate((df_test[category_column_1hot], x_test_std), axis=1)

In [82]:
#Logisticfold交叉驗證 + normalization
from sklearn import preprocessing, linear_model
from sklearn.model_selection import KFold
x = df_train
x = np.array(x)
y = train["is_canceled"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x[train_index], x[test_index], y[train_index], y[test_index]
    logistic_regr = linear_model.LogisticRegression()
    logistic_regr.fit(X_train,y_train)
    y_iscanceled_predict = logistic_regr.predict(X_train)
    accuracy = logistic_regr.score(X_test,y_test)
    print("Logistic Regression Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_logistic_regr = logistic_regr
print("Best Accuracy = ",best_accuracy*100,"%")

c:\users\hg\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy =  85.00027311957174 %


c:\users\hg\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy =  83.65563203321315 %


c:\users\hg\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy =  80.05025674642194 %


c:\users\hg\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy =  79.50398776357478 %
Logistic Regression Accuracy =  77.73953894897848 %
Best Accuracy =  85.00027311957174 %


c:\users\hg\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [84]:
#logistic final accuracy
x_train_new = x
y_train_new = train["is_canceled"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred_is_canceled = best_logistic_regr.predict(x_train_new)
count = 0

for i in range(y_pred_is_canceled.shape[0]):
    if y_pred_is_canceled[i] != y_train_new[i]:
        count += 1
log_accuracy = 1-count/y_pred_is_canceled.shape[0]
print("Logistic Accuracy = ",log_accuracy*100,"%")

Logistic Accuracy =  82.18090045995345 %


In [25]:
#RandomForest adr
from sklearn.ensemble import RandomForestRegressor 
x = df_train
y = train["adr"]
y = np.array(y)
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
rfe = RandomForestRegressor(n_estimators = 100, random_state = 0)
rfe.fit(X_train,y_train)
y_adr_predict_train = rfe.predict(X_train)
y_adr_predict_test = rfe.predict(X_test)
a=lm.score(X_test,y_test)
mse_lm = np.mean((y_adr_predict_train - y_train) ** 2)
print("MSE = ",mse_lm)
print(y_adr_predict_train)
print(y_adr_predict_test)
y_adr_predict = np.concatenate((y_adr_predict_train,y_adr_predict_test),axis=0)
print(y_adr_predict)

MSE =  169.03101722173048
[171.88640336  58.15428785  61.60432304 ... 150.03083603  98.02645162
  33.76355984]
[ 79.47938752  55.72162062  28.53174157 ...  39.18507066 113.8825197
  62.81636739]
[171.88640336  58.15428785  61.60432304 ...  39.18507066 113.8825197
  62.81636739]


In [29]:
a=rfe.score(X_test,y_test)
print(a)

0.7771913342156753


In [36]:
#Neural Network adr
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
x = df_train
y = train["adr"]
y = np.array(y)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)
neural_regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
y_adr_predict_train_nn = neural_regr.predict(X_train)
y_adr_predict_test_nn = neural_regr.predict(X_test)
an = neural_regr.score(X_test, y_test)
mse_nn = np.mean((y_adr_predict_train - y_train) ** 2)
print("MSE = ",mse_nn)
print(y_adr_predict_train_nn)
print(y_adr_predict_test_nn)
y_adr_predict_nn = np.concatenate((y_adr_predict_train_nn,y_adr_predict_test_nn),axis=0)
print(y_adr_predict_nn)

MSE =  4924.0791025435765
[ 71.07830503 143.83314989  73.09887441 ... 103.63514342  73.37627131
  91.66006963]
[68.17805726 52.51756563 90.2975719  ... 86.06455034 47.9007469
 83.50871803]
[ 71.07830503 143.83314989  73.09887441 ...  86.06455034  47.9007469
  83.50871803]


In [86]:
traincolumn = ["is_canceled","arrival_date_year","arrival_date_month","arrival_date_day_of_month","stays_in_weekend_nights","stays_in_week_nights","adr"]
df_train = train[traincolumn]
df_train["is_canceled_predict"] = y_pred_is_canceled
df_train["adr_predict"] = y_adr_predict_nn

date_list = []
month_dict = {"January":"01","February":"02","March":"03","April":"04","May":"05",
             "June":"06","July":"07","August":"08","September":"09","October":"10",
             "November":"11","December":"12"}
for i in range(df_train.shape[0]):
    data = f"{df_train['arrival_date_year'][i]}/{month_dict[df_train['arrival_date_month'][i]]}/{df_train['arrival_date_day_of_month'][i]:02}"
    date_list.append(data)
df_train["arrival_date"] = np.array(date_list)
df_train = df_train.drop(["arrival_date_year","arrival_date_month","arrival_date_day_of_month"],axis=1)
df_train.head()

<ipython-input-86-85aaafe894b3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["is_canceled_predict"] = y_pred_is_canceled
<ipython-input-86-85aaafe894b3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["adr_predict"] = y_adr_predict_nn
<ipython-input-86-85aaafe894b3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,is_canceled,stays_in_weekend_nights,stays_in_week_nights,adr,is_canceled_predict,adr_predict,arrival_date
0,0,0,0,-6.305161,1,71.078305,2015/07/01
1,0,0,2,75.052227,0,143.833150,2015/07/01
2,0,0,2,74.546401,1,73.098874,2015/07/01
3,0,0,2,76.376288,1,18.852033,2015/07/01
4,0,0,2,49.411647,1,72.402406,2015/07/01


In [87]:
adr_list = []
is_canceled_list = []
for i in range(df_train.shape[0]):
    adr_list.append(abs(df_train["adr"][i] - df_train["adr_predict"][i]))
    is_canceled_list.append(abs(df_train["is_canceled"][i] - df_train["is_canceled_predict"][i]))
print("adr Ein = ",sum(adr_list)/df_train.shape[0])
print("is_canceled Ein = ",sum(is_canceled_list)/df_train.shape[0])

adr Ein =  51.21984371260889
is_canceled Ein =  0.17819099540046543


In [88]:
stay_list = []
for i in range(df_train.shape[0]):
    stay = df_train["stays_in_weekend_nights"][i] + df_train["stays_in_week_nights"][i]
    stay_list.append(stay)

df_train["stay"] = np.array(stay_list)
df_train = df_train.drop(["stays_in_weekend_nights","stays_in_week_nights"],axis=1)

df_train = df_train.drop(["adr","is_canceled"],axis=1)

In [89]:
condition = df_train["is_canceled_predict"] == 0
df_train = df_train[condition]
df_train = df_train.reset_index(drop=True)

In [90]:
daily_revenue_list = []
for i in range(df_train.shape[0]):
    daily_revenue = df_train["adr_predict"][i] * df_train["stay"][i]
    daily_revenue_list.append(daily_revenue) 

df_train["daily_revenue"] = np.array(daily_revenue_list)
df_train = df_train.drop(["adr_predict","stay","is_canceled_predict"],axis=1)

In [91]:
drop = []
head = 0
for i in range(1,df_train.shape[0]):
    if df_train["arrival_date"][i] == df_train["arrival_date"][head]: 
        df_train["daily_revenue"][head] = df_train["daily_revenue"][head] + df_train["daily_revenue"][i]
        drop.append(i)
    else:
        head = i
submit = df_train.drop(drop)
submit = submit.reset_index(drop=True)
submit

<ipython-input-91-8081819b65db>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["daily_revenue"][head] = df_train["daily_revenue"][head] + df_train["daily_revenue"][i]


,arrival_date,daily_revenue
0,2015/07/01,17996.811131
1,2015/07/02,16159.331539
2,2015/07/03,11912.006230
3,2015/07/04,12334.262291
4,2015/07/05,21521.156488
...,...,...
635,2017/03/27,34671.189898
636,2017/03/28,28502.600075
637,2017/03/29,34088.611368
638,2017/03/30,41279.277595


In [92]:
submit["label"] = train_label["label"]
predict_list = []
Ein = 0
for i in range(submit.shape[0]):
    predict = submit["daily_revenue"][i]//10000
    predict_list.append(predict)
    #if submit["label"][i] != submit["predict"][i]:
        #Ein+=1
submit["predict"] = np.array(predict_list)
submit

,arrival_date,daily_revenue,label,predict
0,2015/07/01,17996.811131,2.0,1.0
1,2015/07/02,16159.331539,1.0,1.0
2,2015/07/03,11912.006230,1.0,1.0
3,2015/07/04,12334.262291,1.0,1.0
4,2015/07/05,21521.156488,1.0,2.0
...,...,...,...,...
635,2017/03/27,34671.189898,2.0,3.0
636,2017/03/28,28502.600075,1.0,2.0
637,2017/03/29,34088.611368,2.0,3.0
638,2017/03/30,41279.277595,3.0,4.0


In [93]:
Ein = 0
for i in range(submit.shape[0]):
    if submit["label"][i] != submit["predict"][i]:
        Ein+=1
print("Ein = ",Ein/submit.shape[0])

Ein =  0.578125


In [94]:
y_test_iscanceled_predict = best_logistic_regr.predict(df_test)
y_test_adr_predict_nn = neural_regr.predict(df_test)

In [95]:
testcolumn = ["arrival_date_year","arrival_date_month","arrival_date_day_of_month","stays_in_weekend_nights","stays_in_week_nights"]
df_test = test[testcolumn]
df_test["is_canceled_predict"] = y_test_iscanceled_predict
df_test["adr_predict"] = y_test_adr_predict_nn
df_test.head()

<ipython-input-95-a5e8af2a5471>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["is_canceled_predict"] = y_test_iscanceled_predict
<ipython-input-95-a5e8af2a5471>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["adr_predict"] = y_test_adr_predict_nn


,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,is_canceled_predict,adr_predict
0,2017,April,1,2,5,0,109.170372
1,2017,April,1,4,10,1,31.879467
2,2017,April,1,2,5,0,68.487209
3,2017,April,1,2,5,1,108.313450
4,2017,April,1,2,4,1,189.048624


In [96]:
date_list = []
month_dict = {"January":"01","February":"02","March":"03","April":"04","May":"05",
             "June":"06","July":"07","August":"08","September":"09","October":"10",
             "November":"11","December":"12"}
for i in range(df_test.shape[0]):
    data = f"{df_test['arrival_date_year'][i]}-{month_dict[df_test['arrival_date_month'][i]]}-{df_test['arrival_date_day_of_month'][i]:02}"
    date_list.append(data)

df_test["arrival_date"] = np.array(date_list)
df_test = df_test.drop(["arrival_date_year","arrival_date_month","arrival_date_day_of_month"],axis=1)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27859 entries, 0 to 27858
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   stays_in_weekend_nights  27859 non-null  int64  
 1   stays_in_week_nights     27859 non-null  int64  
 2   is_canceled_predict      27859 non-null  int64  
 3   adr_predict              27859 non-null  float64
 4   arrival_date             27859 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 1.1+ MB


<ipython-input-96-762c1d70825e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["arrival_date"] = np.array(date_list)


In [97]:
stay_list = []
for i in range(df_test.shape[0]):
    stay = df_test["stays_in_weekend_nights"][i] + df_test["stays_in_week_nights"][i]
    stay_list.append(stay)

df_test["stay"] = np.array(stay_list)
df_test = df_test.drop(["stays_in_weekend_nights","stays_in_week_nights"],axis=1)

In [98]:
condition = df_test["is_canceled_predict"] == 0
df_test = df_test[condition]
df_test = df_test.reset_index(drop=True)

In [99]:
daily_revenue_list = []
for i in range(df_test.shape[0]):
    daily_revenue = df_test["adr_predict"][i] * df_test["stay"][i]
    daily_revenue_list.append(daily_revenue)

In [100]:
df_test["daily_revenue"] = np.array(daily_revenue_list)
df_test = df_test.drop(["adr_predict","stay","is_canceled_predict"],axis=1)

In [101]:
drop = []
head = 0
for i in range(1,df_test.shape[0]):
    if df_test["arrival_date"][i] == df_test["arrival_date"][head]: 
        df_test["daily_revenue"][head] = df_test["daily_revenue"][head] + df_test["daily_revenue"][i]
        drop.append(i)
    else:
        head = i
submit_test = df_test.drop(drop)
submit_test = submit_test.reset_index(drop=True)
submit_test

<ipython-input-101-29dd57138732>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["daily_revenue"][head] = df_test["daily_revenue"][head] + df_test["daily_revenue"][i]


,arrival_date,daily_revenue
0,2017-04-01,26848.419896
1,2017-04-02,26926.737646
2,2017-04-03,37790.003806
3,2017-04-04,18533.685739
4,2017-04-05,30270.034113
...,...,...
148,2017-08-27,45485.543799
149,2017-08-28,77023.051446
150,2017-08-29,28340.627496
151,2017-08-30,23577.618566


In [102]:
predict_list = []
for i in range(submit_test.shape[0]):
    predict = submit_test["daily_revenue"][i]//10000
    predict_list.append(predict)
submit_test["predict"] = np.array(predict_list)

In [103]:
nolabel = pd.read_csv("C:\\Users\\hg\\Desktop\\MLfinalproject\\test_nolabel.csv")
nolabel

,arrival_date,label
0,4/1/2017,3
1,4/2/2017,2
2,4/3/2017,4
3,4/4/2017,1
4,4/5/2017,4
...,...,...
148,8/27/2017,5
149,8/28/2017,8
150,8/29/2017,4
151,8/30/2017,2


In [104]:
nolabel["label"] = np.array(predict_list)
nolabel

,arrival_date,label
0,4/1/2017,2.0
1,4/2/2017,2.0
2,4/3/2017,3.0
3,4/4/2017,1.0
4,4/5/2017,3.0
...,...,...
148,8/27/2017,4.0
149,8/28/2017,7.0
150,8/29/2017,2.0
151,8/30/2017,2.0


In [105]:
nolabel.to_csv("C:\\Users\\hg\\Desktop\\MLfinalproject\\test_nolabel.csv",index=False)
nolabel["label"].value_counts()

3.0    59
2.0    42
4.0    27
5.0    14
1.0     8
7.0     2
6.0     1
Name: label, dtype: int64